In [ ]:
import torch
###

# During processing we want the largest dimesnion to be the one that is virtual, wheras for loss calculation which is important yet trivial on compute we want the largest possible dimensional surface over which to calulate the loss. the smaller of the two dimensions is the one that should be made virtual

def holographic_transform(input_tensor, timesteps, binary_values=True):
    # input_tensor is a 4D tensor of shape (batch_size, 1, x, y)

    # add a new dimesnsion to the input tensor to make it 5D (b, c, x, y, timesteps)
    input_tensor = input_tensor.reshape(*input_tensor.shape, new_func(timesteps))   #.unsqueeze(4)
    print(input_tensor.shape)

    print("L1", input_tensor)
    print("L2", input_tensor[0].shape)
    print("L3", input_tensor[0][0].shape)
    print("L4", input_tensor[0][0][0].shape)
    print("L5", input_tensor[0][0][0][0].shape)
    print("L6", input_tensor[0][0][0][0][0])


    return output_tensor

def new_func(timesteps):
    return timesteps

# Example usage
input_tensor = torch.tensor([[[[0.5, 0.5, 0.0, 0.0, 0.0],
                            [0.9, 1.0, 0.2, 0.0, 0.0]]]])

print(input_tensor.shape    )
timesteps = 10
binary_hologram = True


print(input_tensor)
output_tensor = holographic_transform(input_tensor, timesteps, binary_hologram)
print(output_tensor)


In [ ]:

print("\n\n")

import matplotlib.pyplot as plt
plt.imshow(input_tensor)
plt.show()

plt.imshow(output_tensor)
plt.show()


In [ ]:
##### WORKING, KEEPS 0.0'S IN THE FIRST DIMENSION AS 1.0 HITS #####


import torch
import torch.nn.functional as F

# Your 2D tensor
input_tensor = torch.tensor([[1.0, 2.0, 0.0],
                             [0.0, 0.0, 0.0],
                             [0.0, 0.0, 3.0]])

def expand_data_to_dim(input_tensor):
    # Determine the number of classes (third dimension size)
    num_classes = int(input_tensor.max()) + 1

    # Convert the input tensor to indices
    indices = input_tensor.long()

    # Create one-hot encoded tensor
    one_hot_encoded = F.one_hot(indices, num_classes=num_classes).float()

    # Permute dimensions to move the new dimension to the front
    print(one_hot_encoded.shape)
    one_hot_encoded = one_hot_encoded.permute(2, 0, 1 )

    print(one_hot_encoded.shape)
    print(one_hot_encoded)

expand_data_to_dim(input_tensor)

In [ ]:
#### WORKING, AUTO REMOVES 0.0 VALUES FROM 3D ARRAY ####

import torch
import torch.nn.functional as F

def expand_data_to_new_dim(input_tensor, timesteps=1000):
    """
    # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc
    
    """

    # QUANITSE VALUES IN THE TENSOR TO steps of (1/timesteps) then multiply by timesteps to arrive at integers (this simplifies to juyt * timestep then round)
    #quantised_tensor = torch.round(input_tensor * timesteps) *100 # / timesteps

    # Determine the number of classes (third dimension size)
    num_classes = timesteps #int(input_tensor.max()) + 1

    # Convert the input tensor to indices
    indices = input_tensor.long()

    # Create a mask for non-zero values
    non_zero_mask = input_tensor != 0

    # Create one-hot encoded tensor
    one_hot_encoded = F.one_hot(indices, num_classes=num_classes).float()
    print(one_hot_encoded.shape)
    # Apply the mask to exclude zero values
    one_hot_encoded = one_hot_encoded * non_zero_mask.unsqueeze(-1)
    print(one_hot_encoded.shape)
    # Permute dimensions to move the new dimension to the front
    one_hot_encoded = one_hot_encoded.permute(0, 1, 4, 2, 3)
    print(one_hot_encoded.shape)
    return one_hot_encoded

# Your 2D tensor
input_tensor = torch.tensor([[[[1.0, 2.0, 0.0],
                             [0.0, 0.0, 0.0],
                             [0.0, 6.0, 3.0]]]])

print(input_tensor.shape)
timesteps = 10    # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc

expanded_tensor = expand_data_to_new_dim(input_tensor, timesteps)
print(expanded_tensor)


In [ ]:
#### QUANITSATION FOR THE HOTSHOT #####

input_tensor = torch.tensor([[0.0, 0.1, 0.0],
                             [0.345, 1.0, 0.0],
                             [0.99999, 0.876, 1.0]])



In [ ]:
class True3DLoss(torch.nn.Module):
    def __init__(self, zero_weighting=1, nonzero_weighting=1, timesteps=1000):
        """
        # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc
        
        """
        super().__init__()   
        self.mse_loss = torch.nn.MSELoss(reduction='mean')
        self.timesteps = timesteps

    def expand_data_to_new_dim(self, input_tensor):
        """
        # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc
        
        """

        # QUANITSE VALUES IN THE TENSOR TO steps of (1/timesteps) then multiply by timesteps to arrive at integers (this simplifies to juyt * timestep then round)
        quantised_tensor = torch.round(input_tensor * self.timesteps) # / timesteps

        # Determine the number of classes (third dimension size)
        num_classes = self.timesteps #int(input_tensor.max()) + 1

        # Convert the input tensor to indices
        indices = input_tensor.long()

        # Create a mask for non-zero values
        non_zero_mask = input_tensor != 0

        # Create one-hot encoded tensor
        one_hot_encoded = F.one_hot(indices, num_classes=num_classes).float()

        # Apply the mask to exclude zero values
        one_hot_encoded = one_hot_encoded * non_zero_mask.unsqueeze(-1)

        # Permute dimensions to move the new dimension to the front
        one_hot_encoded = one_hot_encoded.permute(2, 0, 1)
        
        return one_hot_encoded

    def forward(self, reconstructed_image, target_image):
        reconstructed_3D_view = self.expand_data_to_new_dim(reconstructed_image)
        target_3D_view = self.expand_data_to_new_dim(target_image)
        true3d_loss = self.mse_loss(reconstructed_3D_view, target_3D_view)
        return vy_loss

In [122]:
import torch

import torch

import torch

import torch

import torch

import torch

def expand_data_to_new_dim(input_tensor, timesteps):
    num_classes = timesteps
    indices = input_tensor.long()

    # Create a new dimension for broadcasting
    indices = indices.unsqueeze(2).unsqueeze(3)

    batch_size, channels, T, x, y = indices.size()  # Unpack the correct dimensions
    one_hot_encoded = torch.zeros(batch_size, channels, num_classes, x, y)
    one_hot_encoded.scatter_(2, indices, 1.0)

    return one_hot_encoded


# Your 2D tensor
input_tensor = torch.tensor([[[[1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0]]]])

print(input_tensor.shape)
timesteps = 10    # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc

# Expand the tensor to a 3D tensor#
#input_tensor = expand_data_to_new_dim(input_tensor, timesteps)
#print(input_tensor.shape)

    


torch.Size([1, 1, 3, 3])


In [130]:
import torch
input_tensor = torch.tensor([[[[1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0],
                             
                             [0.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0,1.0, 0.0, 0.0]]]])


input_tensor = (input_tensor * timesteps) -1

input_tensor = torch.where(input_tensor < 0, torch.tensor(0.0), input_tensor)   # could try input_tensor + 1.0 instead of  torch.tensor(0.0)

print(input_tensor)
# Assuming you have 'indices' and 'num_classes' defined
num_classes = 1000
indices = input_tensor.long()

# Reshape the indices tensor for compatibility with scatter_
reshaped_indices = indices.view(indices.size(0), indices.size(1), -1)

# Create a tensor of zeros with the same shape as 'reshaped_indices'
one_hot_encoded = torch.zeros(reshaped_indices.size(0), reshaped_indices.size(1), num_classes, reshaped_indices.size(2))

# Use scatter_ to fill the one-hot encoded tensor
one_hot_encoded.scatter_(2, reshaped_indices.unsqueeze(2), 1)

# Convert the tensor to float
one_hot_encoded = one_hot_encoded.float()
print(one_hot_encoded)

tensor([[[[9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0.],
          [0., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0., 0., 9., 0

In [144]:
class True3DLoss1(torch.nn.Module):
    def __init__(self, timesteps=1000):
        """
        # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc
        
        """
        super().__init__()   
        self.mse_loss = torch.nn.MSELoss(reduction='mean')
        self.timesteps = timesteps


    def expand_data_to_new_dim(self, input_tensor):
        input_tensor = (input_tensor * self.timesteps) - 1

        input_tensor = torch.where(input_tensor < 0, torch.tensor(0.0), input_tensor)   # could try input_tensor + 1.0 instead of  torch.tensor(0.0)

        # Assuming you have 'indices' and 'num_classes' defined
        num_classes =  self.timesteps
        indices = input_tensor.long()

        # Reshape the indices tensor for compatibility with scatter_
        reshaped_indices = indices.view(indices.size(0), indices.size(1), -1)

        # Create a tensor of zeros with the same shape as 'reshaped_indices'
        one_hot_encoded = torch.zeros(reshaped_indices.size(0), reshaped_indices.size(1), num_classes, reshaped_indices.size(2), requires_grad=True)
        
        # Use scatter to fill the one-hot encoded tensor (without in-place operation)
        one_hot_encoded = one_hot_encoded.scatter(2, reshaped_indices.unsqueeze(2), 1)

        # Convert the tensor to float
        one_hot_encoded = one_hot_encoded.float()

        return one_hot_encoded

    def forward(self, reconstructed_image, target_image):
        reconstructed_3D_view = self.expand_data_to_new_dim(reconstructed_image)
        target_3D_view = self.expand_data_to_new_dim(target_image)
        true3d_loss = self.mse_loss(reconstructed_3D_view, target_3D_view)
        return true3d_loss

In [145]:
import torch
import torch.nn.functional as F

class True3DLoss2(torch.nn.Module):
    def __init__(self, timesteps=1000):
        super().__init__()   
        self.timesteps = timesteps
        self.mse_loss = torch.nn.MSELoss(reduction='mean')


    def expand_data_to_new_dim(self, input_tensor):
        input_tensor = (input_tensor * self.timesteps) - 1

        input_tensor = torch.where(input_tensor < 0, torch.tensor(0.0), input_tensor)   # could try input_tensor + 1.0 instead of  torch.tensor(0.0)

        # Assuming you have 'indices' and 'num_classes' defined
        num_classes =  self.timesteps
        indices = input_tensor.long()

        # Reshape the indices tensor for compatibility with scatter_
        reshaped_indices = indices.view(indices.size(0), indices.size(1), -1)

        # Create a tensor of zeros with the same shape as 'reshaped_indices'
        one_hot_encoded = torch.zeros(reshaped_indices.size(0), reshaped_indices.size(1), num_classes, reshaped_indices.size(2), requires_grad=True)
        
        # Use scatter to fill the one-hot encoded tensor (without in-place operation)
        one_hot_encoded = one_hot_encoded.scatter(2, reshaped_indices.unsqueeze(2), 1)

        # Convert the tensor to float
        one_hot_encoded = one_hot_encoded.float()

        return one_hot_encoded

    def forward(self, reconstructed_image, target_image):
        reconstructed_3D_view = self.expand_data_to_new_dim(reconstructed_image)
        target_3D_view = self.expand_data_to_new_dim(target_image)
        true3d_loss = self.mse_loss(reconstructed_3D_view, target_3D_view)
        return true3d_loss


In [146]:
# Create random input tensors
torch.manual_seed(42)
timesteps = 1000
batch_size = 10
height = 128
width = 88
channels = 1

reconstructed_image = torch.rand(batch_size, channels, height, width)
target_image = torch.rand(batch_size, channels, height, width)

# Calculate losses using both versions of the loss function
original_loss_fn = True3DLoss1(timesteps)
modified_loss_fn = True3DLoss2(timesteps)

original_loss = original_loss_fn(reconstructed_image, target_image)
modified_loss = modified_loss_fn(reconstructed_image, target_image)

# Check if the losses are approximately equal within a tolerance
tolerance = 1e-6
assert torch.allclose(original_loss, modified_loss, atol=tolerance), "Losses do not match!"
print("Losses match within the tolerance.")
print("Original loss:", original_loss)
print("Modified loss:", modified_loss)

Losses match within the tolerance.
Original loss: tensor(0.0020, grad_fn=<MseLossBackward0>)
Modified loss: tensor(0.0020)


In [147]:
import timeit

# Create random input tensors
torch.manual_seed(42)
timesteps = 1000
batch_size = 10
height = 32
width = 32
channels = 1

reconstructed_image = torch.rand(batch_size, channels, height, width)
target_image = torch.rand(batch_size, channels, height, width)

# Create instances of the loss functions
original_loss_fn = True3DLoss1(timesteps)
modified_loss_fn = True3DLoss2(timesteps)

# Define a function for timing the original loss function
def time_original_loss():
    original_loss_fn(reconstructed_image, target_image)

# Define a function for timing the modified loss function
def time_modified_loss():
    modified_loss_fn(reconstructed_image, target_image)

# Measure the execution time using timeit
num_iterations = 1000
original_time = timeit.timeit(time_original_loss, number=num_iterations)
modified_time = timeit.timeit(time_modified_loss, number=num_iterations)

print(f"Original Loss Function Time: {original_time:.4f} seconds")
print(f"Modified Loss Function Time: {modified_time:.4f} seconds")

Original Loss Function Time: 84.9020 seconds
Modified Loss Function Time: 120.9007 seconds


In [148]:
import torch
from torch_scatter import scatter_max

src = torch.tensor([[2, 0, 1, 4, 3], [0, 2, 1, 3, 4]])
index = torch.tensor([[4, 5, 4, 2, 3], [0, 0, 2, 2, 1]])

out, argmax = scatter_max(src, index, dim=-1)

print(out)
tensor([[0, 0, 4, 3, 2, 0],
        [2, 4, 3, 0, 0, 0]])

print(argmax)
tensor([[5, 5, 3, 4, 0, 1]
        [1, 4, 3, 5, 5, 5]])


<>:14: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:14: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?


ModuleNotFoundError: No module named 'torch_scatter'

In [167]:
import time


class True3DLoss5(torch.nn.Module):
    def __init__(self, timesteps=1000):
        """
        # Number of dp is related to timestep size. 1,000 = 3dp, 10,000 = 4dp, 100,000 = 5dp, 1,000,000 = 6dp etc
        
        """
        super().__init__()   
        self.mse_loss = torch.nn.MSELoss(reduction='mean')
        self.timesteps = timesteps


    def expand_data_to_new_dim(self, input_tensor):
        input_tensor = (input_tensor * self.timesteps) - 1
        scale_time = time.time() - self.start_time
        input_tensor = torch.where(input_tensor < 0, torch.tensor(0.0), input_tensor)   # could try input_tensor + 1.0 instead of  torch.tensor(0.0)
        where_time = time.time() - self.start_time
        # Assuming you have 'indices' and 'num_classes' defined
        num_classes =  self.timesteps
        indices = input_tensor.long()
        long_tensor_time = time.time() - self.start_time
        # Reshape the indices tensor for compatibility with scatter_
        reshaped_indices = indices.view(indices.size(0), indices.size(1), -1)
        view_time = time.time() - self.start_time
        # Create a tensor of zeros with the same shape as 'reshaped_indices'
        one_hot_encoded = torch.zeros(reshaped_indices.size(0), reshaped_indices.size(1), num_classes, reshaped_indices.size(2), requires_grad=True)
        zeros_time = time.time() - self.start_time
        # Use scatter to fill the one-hot encoded tensor (without in-place operation)
        one_hot_encoded = one_hot_encoded.scatter(2, reshaped_indices.unsqueeze(2), 1)
        scatter_time = time.time() - self.start_time
        # Convert the tensor to float
        one_hot_encoded = one_hot_encoded.float()
        float_time = time.time() - self.start_time
        print(f"scale_time: {scale_time}\n, where_time: {where_time}\n, long_tensor_time: {long_tensor_time}\n, view_time: {view_time}\n, zeros_time: {zeros_time}\n, scatter_time: {scatter_time}\n, float_time: {float_time}\n")

        return one_hot_encoded

    def forward(self, reconstructed_image, target_image):
        start_time = time.time()
        self.start_time = start_time
        reconstructed_3D_view = self.expand_data_to_new_dim(reconstructed_image)
        recon3d_time = time.time() - start_time
        target_3D_view = self.expand_data_to_new_dim(target_image)
        target3d_time = time.time() - start_time
        true3d_loss = self.mse_loss(reconstructed_3D_view, target_3D_view)
        final_time = time.time() - start_time
        print(f"recon3d_time: {recon3d_time}, target3d_time: {target3d_time}, true3d_loss: {true3d_loss}, final_time: {final_time}")

        return true3d_loss

In [168]:
loss_fn = True3DLoss5()

In [169]:
torch.manual_seed(0)
dense_tensor1 = torch.zeros(10, 1, 128, 88)
dense_tensor = torch.zeros(10, 1, 128, 88)
dense_tensor[0, 0, 0, 0] = 1
dense_tensor[0, 0, 1, 1] = 1
dense_tensor[0, 0, 2, 2] = 1

loss = loss_fn(dense_tensor, dense_tensor1)
print(loss)

scale_time: 0.0009996891021728516
, where_time: 1692716077.998831
, long_tensor_time: 0.0019991397857666016
, view_time: 1692716077.998831
, zeros_time: 0.10599875450134277
, scatter_time: 1692716078.0828307
, float_time: 0.10599875450134277

scale_time: 0.20699858665466309
, where_time: 1692716078.0048304
, long_tensor_time: 0.20699858665466309
, view_time: 1692716078.0048304
, zeros_time: 0.3413519859313965
, scatter_time: 1692716078.1438289
, float_time: 0.3413519859313965

final_time: 0.4350404739379883
tensor(5.3267e-08, grad_fn=<MseLossBackward0>)
